### PNS ELEC4 EIEL821 Spectral Analysis 

# Chapter 2: Non-parametric estimation - part 1: periodogram - solutions

This chapter studies the spectral estimation methods based on the periodogram and the correlogram. The first part of this Jupyter notebook focuses on the periodogram. The second part will consider improved periodogram-based methods.

## Periodogram - data length effects on bias and variance

Given an $N$-sample realization $\{y(n)\}_{n = 0}^{N-1}$ of random process $Y(n)$, we can build the zero-padded sequence:

$$y_N(n) = \left\{\begin{array}{lcl} y(n) & & 0 \le n \le N-1\\ 0 & & \text{elsewhere.}\end{array}\right.$$

The **periodogram** can then be computed as

$$\hat{S}_\mathrm{P}(\omega) = \frac 1 N |Y_N(\mathrm{e}^{\jmath\omega})|^2$$

where $Y_N(\mathrm{e}^{\jmath \omega})$ is the discrete-time Fourier transform (DTFT) of sequence $y_N(n)$:

$$Y_N(\mathrm{e}^{\jmath \omega}) = \sum_{n = \infty}^{+\infty} y_N(n)\mathrm{e}^{-\jmath \omega n} = \sum_{n = 0}^{N-1} y(n)\mathrm{e}^{-\jmath \omega n}.$$

**[HAY96, Example 8.2.2, pp. 400-401]** Consider the wide sense stationary random process

$$X(n) = A\sin(w_0n + \phi) + \nu(n)$$ 

where random variable $\phi$ is uniformly distributed in the interval $[0, 2\pi[$ and $\nu(n)$ is a zero-mean unit-variance white noise. We set $A = 5$ and $w_0 = 0.4\pi$ rad/sample.


* Generate 50 realizations of $N = 64$ samples of process $X(n)$. Plot one realization.


* Compute the periodogram (in dB) of each realization using the DTFT-based method recalled above. Superimpose the 50 periodogram realizations in the same figure, with $\omega$ in the interval $[0, \pi]$ rad/sample.


* Compute the average periodogram and plot it (in dB) in a different figure. Compare the average periodogram with the theoretical PSD of the harmonic random process in white noise considered in this exercise.


* Compute and plot (in dB) the variance of the periodogram realizations. Compute the average variance over the whole frequency range.


* Repeat the exercice by increasing the record length to $N = 256$ samples.



*Hint:* Use the discrete Fourier transform (DFT) to compute the DTFT at discrete frequencies $\omega_k = 2\pi/N_\mathrm{FFT}$, $k = 0, 1, \dots, (N_\mathrm{FFT}-1)$. The DFT can be computed in an efficient manner by means of the fast Fourier transform (FFT) algorithm available in $\mathtt{numpy.fft.fft}$. For a fine sampling of the frequency axis, use, e.g., $N_\mathrm{FFT} = 1024$ points in the FFT computation.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

pi = np.pi
fft = np.fft.fft

# ======================================================================
def plot_realization(x):
# Plots one signal realization
#
# x[R, N] : R realizations of N-sample random process

    [R, N] = x.shape
    
    # plot one signal realization
    xabsmax = np.max(abs(x))
    fig = plt.figure(figsize = (10, 4))
    fig.suptitle('A realization of random process X(n), N = ' + str(N) + ' samples')
    plt.stem(range(N), x[1,:])
    plt.xlabel(r'$n$')
    plt.ylabel(r'$x(n)$')
    plt.axis([-1, N, -1.1*xabsmax, 1.1*xabsmax])
    plt.grid()
    
    return

# ======================================================================
def periodogram(x, Nfft = 1024): 
# Computes periodogram of random process realizations
#
# -- Output
# Sx[R, Nfft] : R realizations of spectral estimates over Nfft frequency points in [0, 2*pi] rad/sample
#
# -- Input
# x[R, N] : R realizations of N-sample random process
# Nfft    : number of FFT points
    
    [R, N] = x.shape
    
    print('>>> Periodogram with N = ' + str(N) + ' samples')

    # compute FFT of each realization
    X = fft(x, Nfft, 1)     # note: axes start counting from 0 in Python

    # compute periodogram
    Sx = abs(X)**2/N
    
    return Sx

# ======================================================================
def spectral_bias_variance(Sx, Sx_bounds):
# Computes and plots bias and variance of spectral estimate
#
# Sx[R, Nfft] : R realizations of spectral estimates over Nfft frequency points in [0, 2*pi] rad/sample
# Sx_bounds = [realization_min, realization_max, average_min, average_max]: bounds for spectral representation (dB) 
    
    [R, Nfft] = Sx.shape
    
    # plot periodogram realizations
    ww = np.arange(Nfft/2)*2/Nfft  # sampled frequency axis
    fig = plt.figure(figsize = (12, 4))
    fig.suptitle('Spectral estimate realizations, mean and variance of random process X(n) - ' + info)
    plt.subplot(1, 2, 1)
    
    for i in range(R):
        plt.plot(ww, 10*np.log10(Sx[i, :int(Nfft/2)])) 
    
    plt.xlabel(r'$\omega/\pi$')
    plt.ylabel(r'$S_X(\omega)\ (dB)$')
    plt.axis([0, 1, Sx_bounds[0], Sx_bounds[1]])
    plt.grid()

    # compute average periodogram and variance
    Sx_ave = np.mean(Sx, 0)
    Sx_var = np.var(Sx, 0)
    plt.subplot(1, 2, 2)
    plt.plot(ww, 10*np.log10(Sx_ave[:int(Nfft/2)]))   
    plt.plot(ww, 10*np.log10(Sx_var[:int(Nfft/2)]))   
    plt.xlabel(r'$\omega/\pi$')
    plt.ylabel(r'$S_X(\omega)\ (dB)$')
    plt.legend(('mean', 'variance'))
    plt.axis([0, 1, Sx_bounds[2], Sx_bounds[3]])
    plt.grid()

    # average variance over all frequencies
    print('- average variance over all frequencies =',
                  round(10*np.log10(np.mean(Sx_var)), 2), 'dB')
    print()
    
    return

# ======================================================================

A = 5        # sinusoid amplitude
w0 = 0.4*pi  # sinusoid frequency
R = 50       # number of periodogram realizations
Nfft = 1024  # FFT length
Sx_bounds = [-50, 40, -10, 40] # bounds for spectral representation (dB) [realization_min, realization_max, average_min, average_max]

for N in [64, 256]:      # signal length

    x = np.zeros((R, N))
             
    # generate random realization
    for i in range(R):
       np.random.seed(i)
       x[i, :] = A*np.sin(w0*np.arange(N) + 2*pi*np.random.random()) + np.random.randn(1, N)

    # plot one signal realization
    plot_realization(x)
    
    # compute spectral estimates
    Sx = periodogram(x, Nfft)
    
    # compute and plot bias and variance
    info = 'Periodogram, N = ' + str(N) + ' samples'
    spectral_bias_variance(Sx, Sx_bounds)
    

## Periodogram resolution

**[HAY96, Example 8.2.3, p. 403]** Now consider the random process:

$$X(n) = A\sin(\omega_1 n + \phi_1) + A\sin(\omega_2 n + \phi_2) + \nu(n)$$

where $\omega_1 = 0.4\pi$ rad/sample, $\omega_2 = 0.45\pi$ rad/sample and $A = 5$. Simple calculations (see **[Chapter 2 tutorial]**) show that a record length on the order of $N = 40$ is required to resolve the two sinusoids using the periodogram.

* Repeat the above experiment using $N = 40$ and then $N = 64$. For which sample size can the sinusoids be clearly resolved?


In [ ]:
w1 = 0.4*pi
w2 = 0.45*pi
Sx_bounds = [-40, 50, -10, 50]

for N in [40, 64]:      # total length of signal

    x = np.zeros((R, N))
             
    # generate random realization
    for i in range(R):
       np.random.seed(i)
       x[i, :] = A*np.sin(w1*np.arange(N) + 2*pi*np.random.random()) + A*np.sin(w2*np.arange(N) + 
                                                                                2*pi*np.random.random()) + np.random.randn(1, N)

    # plot one signal realization
    plot_realization(x)
    
    # compute spectral estimates
    Sx = periodogram(x, Nfft)
    
    # compute and plot bias and variance
    info = 'Periodogram, N = ' + str(N) + ' samples'
    spectral_bias_variance(Sx, Sx_bounds)
    

## Periodogram of white noise

**[HAY96, Example 8.2.4, p. 405]** Let $X(n)$ be a zero-mean unit-power white noise. The PSD of this process was obtained in **[Chapter 2 tutorial]**. 

* Repeat the above experiment using $N = 64$ and then $N = 256$. Verify whether the computed mean and variance of the periodogram match the theoretical values found in the tutorial.


In [ ]:
Sx_bounds = [-50, 20, -10, 10]

for N in [64, 256]:      # total length of signal

    x = np.zeros((R, N))
             
    # generate random realization
    for i in range(R):
       np.random.seed(i)
       x[i, :] = np.random.randn(1, N)

    # plot one signal realization
    plot_realization(x)
    
    # compute spectral estimates
    Sx = periodogram(x, Nfft)

    # compute and plot bias and variance
    info = 'Periodogram, N = ' + str(N) + ' samples'
    spectral_bias_variance(Sx, Sx_bounds)
    

## Conclusions

* In the light of the above results, what are the effects of the data record length $N$ on the resolution and variance of the periodogram?
